# Read a shapefile from a zip file

Shapefiles are often packaged as Zip files since the spatial data is actually contained in multiple files associated with the shapefile. You don't have to unzip the zip file before loading the shapefile using Geopandas.

Source: [Reading and Writing Files — GeoPandas 0.9.0 documentation](https://geopandas.org/docs/user_guide/io.html)

In [1]:
# Setup

from pathlib import Path

NOTEBOOK_DIR = Path.cwd()
DATA_DIR = (NOTEBOOK_DIR.parent / "data").absolute()
DATA_DIR_SRC = DATA_DIR / "src"

## Load the shapefile

Note that you can use the `zipinfo` command to view the contents of a zip file:

```bash
zipinfo data/src/grand_blvd.zip
Archive:  data/src/grand_blvd.zip
Zip file size: 4937 bytes, number of entries: 5
drwxr-xr-x  2.1 unx        0 bx stor 19-Jul-05 12:08 grand_blvd/
-rw-r--r--  2.1 unx      371 bX defN 19-Jul-05 12:08 grand_blvd/grand_blvd.dbf
-rw-r--r--  2.1 unx     5404 bX defN 19-Jul-05 12:08 grand_blvd/grand_blvd.shp
-rw-r--r--  2.1 unx      108 bX defN 19-Jul-05 12:08 grand_blvd/grand_blvd.shx
-rw-r--r--  2.1 unx      143 bX defN 19-Jul-05 12:08 grand_blvd/grand_blvd.prj
5 files, 6026 bytes uncompressed, 4109 bytes compressed:  31.8%
```

In newer versions of GeoPandas (which rely on newer versions of [Fiona](https://github.com/Toblerity/Fiona)) you can just specify a specially-formatted path to the zip file.

This file lives in `data/src/grand_blvd.zip`, relative to the root directory of this project.

Within the zip file, there is a subdirectory named `grand_blvd`, which contains the shapefile `grand_blvd.shp`.

In [2]:
!zipinfo ../data/src/grand_blvd.zip

Archive:  ../data/src/grand_blvd.zip
Zip file size: 4937 bytes, number of entries: 5
drwxr-xr-x  2.1 unx        0 bx stor 19-Jul-05 12:08 grand_blvd/
-rw-r--r--  2.1 unx      371 bX defN 19-Jul-05 12:08 grand_blvd/grand_blvd.dbf
-rw-r--r--  2.1 unx     5404 bX defN 19-Jul-05 12:08 grand_blvd/grand_blvd.shp
-rw-r--r--  2.1 unx      108 bX defN 19-Jul-05 12:08 grand_blvd/grand_blvd.shx
-rw-r--r--  2.1 unx      143 bX defN 19-Jul-05 12:08 grand_blvd/grand_blvd.prj
5 files, 6026 bytes uncompressed, 4109 bytes compressed:  31.8%


For this scenario, the argument to `gpd.read_file()` would be this:

In [3]:
# Path to zipped shapefile
zipfile_path = DATA_DIR_SRC / "grand_blvd.zip"
# Path to shapefile within archive
shapefile_path = Path("grand_blvd") / "grand_blvd.shp"

zipfile_url = f"zip://{zipfile_path}!{shapefile_path}"

zipfile_url

'zip:///Users/GHing/workspace/python-data-cheatsheet/data/src/grand_blvd.zip!grand_blvd/grand_blvd.shp'

Note that because the shapefile is in a subdirectory of the zip file, there's a sub-path that appears after the zip file name in this path. The path to the archive and the shapefile within it are separated by a `!`.

In [4]:
import geopandas as gpd
from zipfile import ZipFile

grand_blvd = gpd.read_file(
    zipfile_url
)

grand_blvd

,pri_neigh,sec_neigh,shape_area,shape_len,geometry
0,Grand Boulevard,BRONZEVILLE,4.849250e+07,28196.837157,"POLYGON ((-87.60671 41.81681, -87.60670 41.816..."


In this case, since there's only a single shapefile in the archive, we don't have to specify the full path to the shapefile. We could just provide the path to the containing directory.

In [5]:
shapefile_dir_path = "grand_blvd"

zipfile_url_dir = f"zip://{zipfile_path}!{shapefile_dir_path}"

grand_blvd = gpd.read_file(
    zipfile_url_dir
)

grand_blvd

,pri_neigh,sec_neigh,shape_area,shape_len,geometry
0,Grand Boulevard,BRONZEVILLE,4.849250e+07,28196.837157,"POLYGON ((-87.60671 41.81681, -87.60670 41.816..."


## Method for older versions of Geopandas

With older versions of Geopandas/Fiona, you use the `vfs` keyword argument to specify the path to the zip file that contains the shapefile.

In [6]:
import geopandas as gpd
from zipfile import ZipFile

zipfile_url = f"zip://{zipfile_path}"
grand_blvd = gpd.read_file(
    shapefile_path,
    vfs=zipfile_url,
)

grand_blvd

/Users/GHing/.local/share/virtualenvs/python-data-cheatsheet-ujiHmgN_/lib/python3.9/site-packages/fiona/env.py:408: FionaDeprecationWarning: The vfs keyword argument is deprecated. Instead, pass a URL that uses a zip or tar (for example) scheme.
  return f(*args, **kwargs)


,pri_neigh,sec_neigh,shape_area,shape_len,geometry
0,Grand Boulevard,BRONZEVILLE,4.849250e+07,28196.837157,"POLYGON ((-87.60671 41.81681, -87.60670 41.816..."
